# Vectorización de Usuarios (Multiplicativo)

En este cuaderno, cada estudiante es representado inicialmente por el **vector académico estándar** de su carrera, que refleja el perfil de habilidades definido institucionalmente para esa carrera.

Sin embargo, a diferencia del enfoque aditivo, aquí el vector final del estudiante se ve **netamente influenciado por las asignaturas que considera más relevantes**. Solo las dimensiones asociadas a esas asignaturas relevantes reciben un valor alto (0.99), mientras que el resto de dimensiones se anulan (0). Así, el perfil profesional resultante es una proyección de los intereses y prioridades del propio estudiante, ignorando el resto de la base académica.

## Carga de datos y utilidades

Cargamos los datos de usuarios, la definición de grupos de asignaturas y las funciones auxiliares necesarias para la vectorización y comparación.

In [3]:
import pandas as pd

# Cargar DataFrame y seleccionar/renombrar columnas en un solo paso
df_usuario = (
    pd.read_excel('usuarios/Encuesta recien graduados - pregrado.xlsx')
      .loc[:, [
          'carrera',
          '6.1 ¿Qué asignaturas te resultaron más relevantes durante la carrera? Para completar esta pregunta realizarlo separando por comas: Ejemplo: Estática, Mecánica de materiales, Dinámica'
      ]]
      .rename(columns={
          'carrera': 'Carrera',
          '6.1 ¿Qué asignaturas te resultaron más relevantes durante la carrera? Para completar esta pregunta realizarlo separando por comas: Ejemplo: Estática, Mecánica de materiales, Dinámica': 'Asignaturas Relevantes'
      })
)

df_usuario.head()

,Carrera,Asignaturas Relevantes
0,(RRA20) INGENIERÍA CIVIL,Hidráulica
1,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,"Control industrial, PLCs, instalaciones eléctr..."
2,(RRA20) SOFTWARE,"Programación, aplicaciones móviles, estructura..."
3,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,Robótica y todo lo relacionado a PLCs
4,(RRA20) DESARROLLO DE SOFTWARE,"Bases de datos, Desarrollo de aplicaciones web..."


## Lógica multiplicativa: vectorización personalizada del usuario

Para cada usuario, creamos un vector de 69 dimensiones donde:
- Las posiciones correspondientes a las asignaturas relevantes se establecen en 0.99.
- El resto de posiciones se ponen en 0.

Esto se realiza ignorando completamente el vector académico base.

In [4]:
import pickle

# Cargar los vectores académicos procesados
with open('datos_procesados.pkl', 'rb') as f:
    datos = pickle.load(f)
tfidf_epn_69d = datos['tfidf_epn_69d']

# Diccionario de mapeo manual entre carreras reportadas y carreras académicas vectorizadas
mapeo_carreras = {
    '(RRA20) COMPUTACIÓN': 'Ingenieria En Ciencias De La Computacion',
    '(RRA20) AGROINDUSTRIA': 'Ingenieria Agroindustria',
    '(RRA20) ADMINISTRACIÓN DE EMPRESAS': 'Licenciatura Administracion De Empresas',
    '(RRA20) INGENIERÍA AMBIENTAL': 'Ingenieria Ambiental',
    '(RRA20) ECONOMÍA': 'Economia', # Se repite
    'INGENIERIA EN CIENCIAS ECONOMICAS Y FINANCIERAS':'Economia', # Se repite
    '(RRA20) ELECTRICIDAD': 'Ingenieria En Electricidad',
    '(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN': 'Ingenieria En Electronica Y Automatizacion',
    '(RRA20) FÍSICA': 'Fisica',
    'FISICA': 'Fisica',
    '(RRA20) GEOLOGÍA': 'Ingenieria En Geologia', # Se repite
    'INGENIERIA GEOLOGICA':'Ingenieria En Geologia', # Se repite
    '(RRA20) INGENIERÍA DE LA PRODUCCIÓN': 'Ingenieria De La Produccion',
    '(RRA20) MATEMÁTICA': 'Matematica',
    '(RRA20) MECÁNICA': 'Ingenieria En Mecanica', # Se repite
    'INGENIERIA MECANICA':'Ingenieria En Mecanica', # Se repite
    '(RRA20) PETRÓLEOS':'Ingenieria En Petroleos',
    '(RRA20) INGENIERÍA QUÍMICA': 'Ingenieria Quimica',
    '(RRA20) DESARROLLO DE SOFTWARE': 'Ingenieria En Software', # Se repite
    '(RRA20) SOFTWARE': 'Ingenieria En Software', # Se repite
    '(RRA20) TELECOMUNICACIONES':'Ingenieria En Telecomunicaciones',
    '(RRA20) INGENIERÍA CIVIL': 'Ingenieria Civil'

    # CARRERAS ADICIONALES NO CONTEMPLADAS POR LA ENCUESTA
    # (RRA20) AGUA Y SANEAMIENTO AMBIENTAL
    # (RRA20) ELECTROMECÁNICA
    # INGENIERIA EMPRESARIAL
    # (RRA20) REDES Y TELECOMUNICACIONES
    # (RRA20) TECNOLOGÍAS DE LA INFORMACIÓN

    # CARRERAS ADICIONALES NO CONTEMPLADAS POR EL CORPUS ACADÉMICO
    # 'Matematica Aplicada',
    # 'Ingenieria En Materiales',
    # 'Ingenieria En Mecatronica',
    # 'Ingenieria En Sistemas De Informacion',
    # 'Ingenieria En Telecomunicacion De La Informacion',
    # 'Seguridad De Redes De Informacion',
}

# Asignar la carrera académica vectorizada a cada usuario
df_usuario['Carrera Asignada'] = df_usuario['Carrera'].map(mapeo_carreras)

# Asignar el vector académico correspondiente (como array)
df_usuario['Vector Academico'] = df_usuario['Carrera Asignada'].apply(
    lambda x: tfidf_epn_69d.T.loc[x].values if pd.notnull(x) else None
)

# Visualizar las primeras filas para verificar la asignación
df_usuario[['Carrera', 'Carrera Asignada']].head(10)

,Carrera,Carrera Asignada
0,(RRA20) INGENIERÍA CIVIL,Ingenieria Civil
1,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,Ingenieria En Electronica Y Automatizacion
2,(RRA20) SOFTWARE,Ingenieria En Software
3,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,Ingenieria En Electronica Y Automatizacion
4,(RRA20) DESARROLLO DE SOFTWARE,Ingenieria En Software
5,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,Ingenieria En Electronica Y Automatizacion
6,(RRA20) DESARROLLO DE SOFTWARE,Ingenieria En Software
7,(RRA20) INGENIERÍA CIVIL,Ingenieria Civil
8,(RRA20) INGENIERÍA DE LA PRODUCCIÓN,Ingenieria De La Produccion
9,(RRA20) INGENIERÍA DE LA PRODUCCIÓN,Ingenieria De La Produccion


### Personalización del vector académico según asignaturas relevantes

A continuación, se ajusta el vector académico de cada estudiante según las asignaturas que considera más relevantes. Si un estudiante menciona una asignatura relacionada con una habilidad técnica (por ejemplo, "Redes"), la dimensión correspondiente en su vector se pondera a 0.99 y las demás se cambiaran por 0.

In [5]:
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Usar las dimensiones (habilidades técnicas) como referencia
habilidades = list(tfidf_epn_69d.T.columns)

# Vectorizador TF-IDF para similitud semántica entre asignaturas y habilidades
vectorizer = TfidfVectorizer().fit(habilidades)

# Función para normalizar y tokenizar asignaturas
def normalizar_texto(texto):
    if pd.isnull(texto):
        return []
    asignaturas = [a.strip().lower() for a in re.split(r',|;|/|\n', str(texto)) if a.strip()]
    return asignaturas

# Función para encontrar la habilidad más similar a cada asignatura relevante
def encontrar_habilidades_similares(asignatura, habilidades, vectorizer, threshold=0.5):
    vect_asig = vectorizer.transform([asignatura])
    vect_habs = vectorizer.transform(habilidades)
    sims = cosine_similarity(vect_asig, vect_habs).flatten()
    indices = np.where(sims >= threshold)[0]
    return [habilidades[i] for i in indices]

# Asignar ponderación SOLO a las dimensiones relevantes (multiplicativo)
def vector_multiplicativo(asignaturas_relevantes, habilidades, vectorizer, valor=0.99):
    vector = np.zeros(len(habilidades))
    asignaturas = normalizar_texto(asignaturas_relevantes)
    for asignatura in asignaturas:
        habilidades_similares = encontrar_habilidades_similares(asignatura, habilidades, vectorizer, threshold=0.5)
        for hab in habilidades_similares:
            idx = habilidades.index(hab)
            vector[idx] = valor
    return vector

# Crear el nuevo vector personalizado para cada estudiante (multiplicativo)
df_usuario['Vector Estudiante Profesional Multiplicativo'] = df_usuario.apply(
    lambda row: vector_multiplicativo(row['Asignaturas Relevantes'], habilidades, vectorizer)
    if isinstance(row['Vector Academico'], np.ndarray) else None,
    axis=1
)

df_usuario[['Carrera', 'Asignaturas Relevantes', 'Vector Estudiante Profesional Multiplicativo']].head()

,Carrera,Asignaturas Relevantes,Vector Estudiante Profesional Multiplicativo
0,(RRA20) INGENIERÍA CIVIL,Hidráulica,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,"Control industrial, PLCs, instalaciones eléctr...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,(RRA20) SOFTWARE,"Programación, aplicaciones móviles, estructura...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,Robótica y todo lo relacionado a PLCs,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,(RRA20) DESARROLLO DE SOFTWARE,"Bases de datos, Desarrollo de aplicaciones web...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.99,..."


In [7]:
import numpy as np

# Función para comparar vectores y obtener las dimensiones alteradas
def diferencias_vectoriales(vec1, vec2):
    if vec1 is None or vec2 is None:
        return []
    return np.where(np.abs(vec1 - vec2) > 1e-6)[0]  # Índices donde hay diferencia

# Aplicar comparación a cada fila
df_usuario['Dimensiones Modificadas'] = df_usuario.apply(
    lambda row: diferencias_vectoriales(row['Vector Academico'], row['Vector Estudiante Profesional Multiplicativo'])
    if isinstance(row['Vector Academico'], np.ndarray) and isinstance(row['Vector Estudiante Profesional Multiplicativo'], np.ndarray) else [],
    axis=1
)

# Filtrar estudiantes con al menos una dimensión modificada
df_modificados = df_usuario[df_usuario['Dimensiones Modificadas'].apply(len) > 0]

print(f"Cantidad de estudiantes con vector modificado: {len(df_modificados)}")

# Mostrar algunos ejemplos de cambios
for idx, row in df_modificados.head(5).iterrows():
    print(f"Estudiante {idx} | Carrera: {row['Carrera']} | Dimensiones modificadas: {row['Dimensiones Modificadas']}")
    print("Asignaturas relevantes:", row['Asignaturas Relevantes'])
    print("Dimensiones alteradas:", [dim for i, dim in enumerate(tfidf_epn_69d.T.columns) if i in row['Dimensiones Modificadas']])
    print("-")

Cantidad de estudiantes con vector modificado: 328
Estudiante 0 | Carrera: (RRA20) INGENIERÍA CIVIL | Dimensiones modificadas: [ 0 28 32 51 52 61]
Asignaturas relevantes: Hidráulica
Dimensiones alteradas: ['administración de empresas, gestión de calidad, gestión de datos ...', 'estructura de datos, estructuras', 'geofísica, geomecánica, geotecnia', 'optimización, optimización de procesos', 'producción', 'tratamiento de aguas, tratamientos térmicos']
-
Estudiante 1 | Carrera: (RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN | Dimensiones modificadas: [14 15 26 36 51 52]
Asignaturas relevantes: Control industrial, PLCs, instalaciones eléctricas y potencia.
Dimensiones alteradas: ['control de plagas', 'control de procesos', 'electrónica, electrónica analógica, electrónica digital ...', 'instalaciones eléctricas, mediciones eléctricas, protecciones eléctricas', 'optimización, optimización de procesos', 'producción']
-
Estudiante 2 | Carrera: (RRA20) SOFTWARE | Dimensiones modificadas: [19 28 35 47 57]

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd
import numpy as np
import os
import pickle

# Cargar grupos si no existen
if 'grupos_bge_ngram' not in globals():
    with open('datos_procesados.pkl', 'rb') as f:
        datos = pickle.load(f)
    grupos_bge_ngram = datos['grupos_bge_ngram']

# Configuración de carreras
CARRERAS_CONFIG = [
    ('Ingenieria En Ciencias De La Computacion', 'Computación', 'todas_las_plataformas/Computación/Computación_Merged.csv'),
    ('Ciencias De Datos E Inteligencia Artificial', 'Ciencias De Datos E IA', 'todas_las_plataformas/Ciencia_de_Datos/Ciencia_de_Datos_Merged.csv'),
    ('Ingenieria En Software', 'Software', 'todas_las_plataformas/Software/Software_Merged.csv'),
    ('Ingenieria En Sistemas De Informacion', 'Sistemas de Información', 'todas_las_plataformas/Sistemas_de_Información/Sistemas_de_Información_Merged.csv'),
    ('Licenciatura Administracion De Empresas', 'Administración de Empresas', 'todas_las_plataformas/Administración_de_Empresas/Administración_de_Empresas_Merged.csv'),
    ('Ingenieria Agroindustria', 'Agroindustria', 'todas_las_plataformas/Agroindustria/Agroindustria_Merged.csv'),
    ('Matematica', 'Matemática', 'todas_las_plataformas/Matemática/Matemática_Merged.csv'),
    ('Matematica Aplicada', 'Matemática Aplicada', 'todas_las_plataformas/Matemática_Aplicada/Matemática_Aplicada_Merged.csv'),
    ('Fisica', 'Física', 'todas_las_plataformas/Física/Física_Merged.csv'),
    ('Ingenieria En Geologia', 'Geología', 'todas_las_plataformas/Geología/Geología_Merged.csv'),
    ('Ingenieria De La Produccion', 'Ingeniería De La Producción', 'todas_las_plataformas/Ingeniería_de_la_Producción/Ingeniería_de_la_Producción_Merged.csv'),
    ('Ingenieria En Materiales', 'Materiales', 'todas_las_plataformas/Materiales/Materiales_Merged.csv'),
    ('Ingenieria En Mecanica', 'Mecánica', 'todas_las_plataformas/Mecánica/Mecánica_Merged.csv'),
    ('Ingenieria En Mecatronica', 'Mecatrónica', 'todas_las_plataformas/Mecatrónica/Mecatrónica_Merged.csv'),
    ('Ingenieria En Petroleos', 'Petróleos', 'todas_las_plataformas/Petróleos/Petróleos_Merged.csv'),
    ('Ingenieria Quimica', 'Ingeniería Química', 'todas_las_plataformas/Ingeniería_Química/Ingeniería_Química_Merged.csv'),
    ('Ingenieria En Telecomunicaciones', 'Telecomunicaciones', 'todas_las_plataformas/Telecomunicaciones/Telecomunicaciones_Merged.csv'),
    ('Ingenieria Civil', 'Ingeniería Civil', 'todas_las_plataformas/Ingeniería_Civil/Ingeniería_Civil_Merged.csv'),
    ('Economia', 'Economía', 'todas_las_plataformas/Economía/Economía_Merged.csv'),
    ('Ingenieria En Electricidad', 'Electricidad', 'todas_las_plataformas/Electricidad/Electricidad_Merged.csv'),
    ('Ingenieria En Electronica Y Automatizacion', 'Electrónica Y Automatización', 'todas_las_plataformas/Electrónica_y_Automatización/Electrónica_y_Automatización_Merged.csv'),
    ('Ingenieria Ambiental', 'Ingeniería Ambiental', 'todas_las_plataformas/Ingeniería_Ambiental/Ingeniería_Ambiental_Merged.csv'),
]

carrera_to_csv = {c[0]: c[2] for c in CARRERAS_CONFIG}

# Vectorización de ofertas
def vectorizar_ofertas_csv(ruta_csv, habilidades, grupos_bge_ngram):
    if not os.path.exists(ruta_csv):
        return None, None
    df = pd.read_csv(ruta_csv, dtype=str)
    if not {'skills', 'description'}.issubset(df.columns):
        return None, None
    textos = (
        df[['skills', 'description']]
        .fillna('')
        .agg(' '.join, axis=1)
        .str.lower()
        .tolist()
    )
    if not textos:
        return None, None
    # CountVectorizer
    vectorizer = CountVectorizer(
        vocabulary=habilidades,
        analyzer='word',
        ngram_range=(1, 5),
        lowercase=True
    )
    X = vectorizer.transform(textos)
    matriz_td = pd.DataFrame(
        X.T.toarray(),
        index=vectorizer.get_feature_names_out()
    )
    # Agrupación segura (sin KeyError)
    matriz_69d = pd.DataFrame(
        0,
        index=grupos_bge_ngram.keys(),
        columns=range(len(textos))
    )
    for label, terms in grupos_bge_ngram.items():
        terms_validos = [t for t in terms if t in matriz_td.index]
        if terms_validos:
            matriz_69d.loc[label] = matriz_td.loc[terms_validos].sum(axis=0)
    # TF-IDF
    tfidf = TfidfTransformer(norm='l2')
    tfidf_69d = tfidf.fit_transform(matriz_69d.values)
    tfidf_69d = pd.DataFrame(
        tfidf_69d.toarray(),
        index=matriz_69d.index,
        columns=matriz_69d.columns
    ).T
    return tfidf_69d, df

# Ranking de ofertas por estudiante usando ÁNGULO COSENO (grados)
for idx, row in df_usuario.head(2).iterrows():
    carrera = row['Carrera Asignada']
    vector_estudiante = row['Vector Estudiante Profesional Multiplicativo']
    ruta_csv = carrera_to_csv.get(carrera)
    if not ruta_csv or not isinstance(vector_estudiante, np.ndarray):
        continue
    tfidf_ofertas_69d, df_ofertas = vectorizar_ofertas_csv(
        ruta_csv, habilidades, grupos_bge_ngram
    )
    if tfidf_ofertas_69d is None or tfidf_ofertas_69d.empty:
        continue
    vector_estudiante_69d = vector_estudiante.reshape(1, -1)
    # Cosine similarity (mayor = más similar)
    sims = cosine_similarity(
        vector_estudiante_69d,
        tfidf_ofertas_69d.values
    )[0]
    # Ángulo coseno en grados
    from numpy import arccos, clip, degrees
    angles_rad = arccos(clip(sims, -1, 1))
    angles_deg = degrees(angles_rad)
    top_idx = sims.argsort()[::-1][:5]
    print(f"\nEstudiante {idx} | Carrera: {carrera}")
    print("Top 5 ofertas laborales más similares:")
    for i in top_idx:
        print(f"Oferta {i} | Similitud: {sims[i]:.10f} | Ángulo: {angles_deg[i]:.2f}°")
        print(f"  - HARD skills: {df_ofertas.iloc[i]['skills']}")
        print(f"  - EURACE skills: {df_ofertas.iloc[i]['EURACE_skills']}")
        print(f"  - Cargo a Desempeñar: {df_ofertas.iloc[i]['job_title']}")
        print(f"  - Description: {df_ofertas.iloc[i]['description'][:50]}...")



Estudiante 0 | Carrera: Ingenieria Civil
Top 5 ofertas laborales más similares:
Oferta 3863 | Similitud: 0.0000000000 | Ángulo: 90.00°
  - HARD skills: Gestión de proyectos, Ingeniería de construcción, Ingeniería civil, Infraestructura educativa, Proyectos de construcción, Colaboración en equipo, Resolución de problemas, Comunicación, Planificación, Presupuestación, Control de calidad, Gestión de seguridad, Cumplimiento regulatorio, Relaciones con el cliente, Gestión del tiempo, Habilidades técnicas
  - EURACE skills: Gestión, Comunicación efectiva, Trabajo en equipo, Ética profesional
  - Cargo a Desempeñar: Profesional de Proyectos de Infraestructura Educacional- Calama
  - Description: Company Description: DRS Ingeniería y Gestión

J...
Oferta 1327 | Similitud: 0.0000000000 | Ángulo: 90.00°
  - HARD skills: nan
  - EURACE skills: Trabajo en equipo
  - Cargo a Desempeñar: Licensed Civil Engineer
  - Description: &nbsp;...Job Description 
 Join our collaborative...
Oferta 1295 | Simi